<h1>Welcome to the Resource Watch Duplicate Carto Layers Tool!</h1>

In [ ]:
#Import necessary layers
#!pip install LMIPy
import LMIPy as lmi
import os
import json
import geopandas as gpd
from IPython.display import display
from pprint import pprint
from datetime import datetime
import numpy as np
import json
import dotenv
#insert the location of your .env file here:

<font color=red>**ACTION REQUIRED** <br>
<font color=blue>**Enter your API Token Below**</font>

In [ ]:
#API token needed to make changes
API_TOKEN = os.getenv('RW_API_KEY')

<font color=red>**ACTION REQUIRED** <br>
<font color=blue>**Enter your dataset ID below** <br>
The dataset will be printed out below, then the first layer in the list of layers, followed by the number of layers in the dataset. We will duplicate the first layer.</font>

In [ ]:
#Select Dataset and Layer to Clone, here we just chose the first layer
dataset_id = '' #NEEDS TO BE THE LONG ONE LIKE 6cfd6255-609e-4922-8709-5aaad0db9fae
dataset = lmi.Dataset(dataset_id) #select the dataset by changing the ID
long_dataset_id = dataset.id

print('Dataset selected is below: ')
display(dataset)

layer_to_clone = dataset.layers[0]   #Chose the first layer

print('First layer is below:')
display(layer_to_clone)

print("Number of layers in dataset: {}".format(len(dataset.layers)))


__________________________________________________________________________________________________________________
<font color=blue>**Now we will get the attributes that need to change**<br>
This includes the layer name, description, SQL statement, timeline order, timeline label, and interaction <br>
We'll first print them out so you can see</font>

In [ ]:
#Gathering the right attributes that need to change
name = layer_to_clone.attributes['name']
description = layer_to_clone.attributes['description']
appConfig = layer_to_clone.attributes['layerConfig']
sql = appConfig['source']['provider']['layers'][0]['options']['sql']
order = str(appConfig['order'])
timeLineLabel = appConfig['timelineLabel']
interactionConfig = layer_to_clone.attributes['interactionConfig']

#Print them out
print('Name: \n{} \n'.format(name))
print('Description: \n{} \n'.format(description))
print('SQL Statement: \n{} \n'.format(sql))
print('Order Value: \n{} \n'.format(order))
print('Timeline Label: \n{} \n'.format(timeLineLabel))
print('Interaction Config: \n{} \n'.format(interactionConfig))

<h1>Here we will replace the values in the attributes and decide what to loop over.</h1>

<font color=red>**ACTION REQUIRED**</font> <br>
<font color=blue>**Enter the text you would like to replace in the attributes above in the variable 'replace_string'** <br>
**AND enter the list of values you'd like to replace the search string with in the variable 'loop_values'.**<br>
The next cell will search for that string and replace it with a {} so it can be filled in later.<br>
Then we will loop over the values in 'loop_values' and replace the {} in the attributes with the values.</font>

In [ ]:
#Replace this value with the string you'd like to replace in the attributes
replace_string = '' #e.g., 1990

#Enter the list you want to loop over to create new layers
loop_values = np.arange(1951,2021)

#Print out the values so you can double check them
print('Replace String: {}'.format(replace_string))
print('Loop values: {}'.format(loop_values))

<font color=blue>**Above are the values you entered. Double check that they look okay.**</font>

<font color=blue>**Next we will print out the attributes after replacing the 'replace_string' with '{}'**</font>

In [ ]:
name_convention = name.replace(replace_string,'{}')
description_convention = description.replace(replace_string,'{}')
sql_convention = sql.replace(replace_string,'{}')
order_convention = order.replace(replace_string,'{}')
timeLineLabel_convention = timeLineLabel.replace(replace_string,'{}')
for i, dictionary in enumerate(interactionConfig.get('output')):
    for key,value in dictionary.items():
        if value != None:
            if replace_string in value:
                print(key)
                print(i)
                print(interactionConfig.get('output')[i])
                interactionConfig.get('output')[i][key] = value.replace(replace_string,'{}')
print('Name: \n{} \n'.format(name_convention))
print('Description: \n{} \n'.format(description_convention))
print('SQL Statement: \n{} \n'.format(sql_convention))
print('Order Value: \n{} \n'.format(order_convention))
print('Timeline Label: \n{} \n'.format(timeLineLabel_convention))
print('Interaction Config: \n{} \n'.format(interactionConfig))

<font color=blue>**Check out the values above, do they look okay? The places with a '{}' will be replaced when we loop over the values in 'loop_values'**</font>

In [ ]:
#If any of the variables above are not okay, replace it in this cell

<h1>Now we will loop through and create new layers!</h1>

<font color=blue>**The cell below loops through the values in 'loop_values' and replaces all of the {} with the value in each of the attributes. It will skip over a layer if a layer with that name already exists. When running the cell bellow, it sometimes returns an error that it times out, just run the cell again.**</font>

In [ ]:
#SOMETIMES WE GET AN ERROR THAT IT TIMES OUT, JUST RUN THE CELL AGAIN

#Load in dataset again to have the most updated version
dataset = lmi.Dataset(dataset_id)

#Get the names of all the layers
layers = dataset.layers
layer_names = list(map(lambda x: x.attributes['name'], layers))

#Loop over loop values
for new_value in loop_values:
    #Get layer name
    new_layer_name = name_convention.replace('{}',str(new_value))  
    
    #Check that layer does not already exist
    if new_layer_name not in layer_names:
        
        #If it doesn't, set the new values
        new_description = description_convention.replace('{}',str(new_value))
        new_sql = sql_convention.replace('{}',str(new_value))
        new_timeline_label = timeLineLabel_convention.replace('{}',str(new_value))
        try: 
            new_order = int(order_convention.replace('{}',str(new_value)))
        except:
            new_order = int(order)
        
        #Print those bad boys out
        print('NEW LAYER NAME: "{}"'.format(new_layer_name))
        print('New DESCRIPTION: "{}"'.format(new_description))
        print('NEW SQL: "{}"'.format(new_sql))
        print('NEW TIMELINE LABEL: "{}"'.format(new_timeline_label))
        print('NEW ORDER VALUE: "{}"'.format(new_order))
        
        #Clone layer
        clone_attributes = {
            'name': new_layer_name,
            'description': new_description
            }
        new_layer = layer_to_clone.clone(token=API_TOKEN, env='production', layer_params=clone_attributes, 
                     target_dataset_id=dataset_id)
        #Replace layer attributes with new values
        appConfig = new_layer.attributes['layerConfig']
        appConfig['source']['provider']['layers'][0]['options']['sql'] = new_sql
        appConfig['order'] = new_order
        appConfig['timelineLabel'] = new_timeline_label
        payload = {
        'layerConfig': {
            **appConfig
            }
        }
        new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
        
        #Replace interaction config with new values
        interactionConfig = new_layer.attributes['interactionConfig']
        for i,element in enumerate(interactionConfig['output']):
            if '{}' in element.get('property'):
                interactionConfig['output'][i]['property'] = interactionConfig['output'][i]['property'].replace('{}',str(new_value))
        payload = {
        'interactionConfig': {
            **interactionConfig
            }
        }
        new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
        
        #Replace layer name and description
        payload = {
            'name': new_layer_name,
            'description': new_description
        }
        new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
        print(new_layer)
        print('\n')
        
    else:
        print('Layer "{}" already exists!'.format(new_layer_name))
        
        
        

In [ ]:
#Use this section to delete layers if they are stuck in staging or you want to bulk delete layers

# #UNCOMMENT THIS IF YOU WANT TO DELETE A SPECIFIC LAYER
# #Select layer by layer id:
# layer = lmi.Layer('c3075c5a-5567-4b09-bc0d-96ed1673f8b6')
# layer.delete(token=API_TOKEN)

# #UNCOMMENT THIS SECTION IF YOU WANT TO DELETE ALL LAYERS EXCEPT THE FIRST ONE
# #Enter dataset id (the long one)
# dataset_id = 'e32bdaba-5a72-487c-b640-668adaea2976' #THIS NEEDS TO BE THE LONG STRING ONE
# dataset = lmi.Dataset(dataset_id) 
# for i, layer in enumerate(dataset.layers):
#     if i<len(dataset.layers)-1:
#         display(layer)
#         layer.delete(token=API_TOKEN)